In [19]:
#! pip install -q langchain-openai langchain langchain-text-splitters lxml octoai-sdk pymilvus
#! pip install python-dotenv

In [20]:
#! pip install langchain_text_splitters
#! pip install lxml

In [21]:
from dotenv import load_dotenv
import os
#from config import OCTOAI_API_TOKEN

load_dotenv()
#OPENAI_API_KEY = os.getenv["OPENAI_API_KEY"]
OCTOAI_API_TOKEN = os.getenv("OCTOAI_API_TOKEN")

In [22]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, HTMLHeaderTextSplitter

url = "https://en.wikipedia.org/wiki/Friends"

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
    ("div", "Divider")
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# for local file use html_splitter.split_text_from_file(<path_to_file>)
html_header_splits = html_splitter.split_text_from_url(url)

In [23]:
chunk_size = 1024
chunk_overlap = 128
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
)

# Split
splits = text_splitter.split_documents(html_header_splits)

In [24]:
from langchain_community.vectorstores import Milvus

In [25]:
from langchain_community.embeddings import OctoAIEmbeddings
from langchain_community.llms.octoai_endpoint import OctoAIEndpoint
llm = OctoAIEndpoint(
        model="llama-2-13b-chat-fp16",
        max_tokens=1024,
        presence_penalty=0,
        temperature=0.1,
        top_p=0.9,
        
    )
embeddings = OctoAIEmbeddings(endpoint_url="https://text.octoai.run/v1/embeddings")

/Users/eviezheng/Documents/GitHub/LLM-hackthon-with-OctoAI/venv/lib/python3.11/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! model is not default parameter.
                model was transferred to model_kwargs.
                Please confirm that model is what you intended.
  warnings.warn(


In [26]:
! pip install pymilvus

In [27]:
vector_store = Milvus.from_documents(
    splits,
    embedding=embeddings,
    connection_args={"host": "localhost", "port": 19530},
    collection_name="starwars"
)

In [28]:
retriever = vector_store.as_retriever()

In [29]:
from langchain.prompts import ChatPromptTemplate
template="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

In [30]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [31]:
chain.invoke("Who is Monica's Actor?")

" Monica's actor is Courteney Cox."

In [32]:
from langchain.prompts import ChatPromptTemplate
template="""You are a literary critic. You are given some context and asked to answer questions based on only that context.
Question: {question} 
Context: {context} 
Answer:"""
lit_crit_prompt = ChatPromptTemplate.from_template(template)

In [33]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
lcchain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | lit_crit_prompt
    | llm
    | StrOutputParser()
)

In [34]:
from pprint import pprint

pprint(lcchain.invoke("What is the Monica's actor,tell her story"))

(' Monica is a talented chef who is promoted to head chef at the Iridium '
 'restaurant, but she is later fired for accepting gifts from a supplier. She '
 'is forced to take an embarrassing job as a waitress at a 1950s-style diner '
 'and begins dating Richard Burke, a recently divorced family friend who is 21 '
 'years her senior. However, their relationship ends when Monica realizes that '
 "Richard does not want more children. Joey, one of Monica's friends, is cast "
 'in a fictional version of the soap opera, Days of Our Lives, as neurosurgeon '
 "Dr. Drake Ramoray, and he moves out of his and Chandler's apartment, forcing "
 'Chandler to get a new roommate. Throughout the first season, the writers '
 "adjusted the characters to suit the actors, and Monica's neuroses become a "
 'key aspect of her character.')
